In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from pyproj import Transformer
import seaborn as sns
import networkx as nx
from scipy.spatial import Delaunay
from itertools import combinations
import matplotlib.pyplot as plt

from road_utils import *
from city_utils import plotMapWithColors
from useful_methods.data_processing import extract_data
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from useful_methods.neighbours_delaunay.miscellaneous_for_neighbouring import mean_distance_choice, mean_distance_to_NN
from useful_methods.ihm.maps.mapUtils import *
from useful_methods.neighbours_delaunay.enhanced_criteria import km_distance
from useful_methods.neighbours_delaunay.simple_criteria import distance_criterion, angle_criterion


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', techno='4g')#, region='Normandie') #Occitanie, Île-de-France
G, pos = delaunay_graph(df_extracted)

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
cityStationsLabels = []

for station in mean_distances.index:
    if(mean_distances[station]<=1):
        cityStationsLabels.append(station)

In [ ]:
cityLabels = pd.Series(DBSCAN(eps=4000, min_samples=1).fit(df_extracted[['x','y']].loc[cityStationsLabels]).labels_, index = cityStationsLabels)

In [ ]:
cityBsStationCount = cityLabels.value_counts()

bigCitiesClusters = cityBsStationCount[cityBsStationCount >= 27].index
littleCitiesClusters = cityBsStationCount[cityBsStationCount < 27].index


In [ ]:
cityCenters = {cityLabel : (0,0) for cityLabel in cityLabels.unique()}
for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].loc[cityStationsLabels].itertuples():
    coords = cityCenters.get(cityLabels[bs_id])
    cityCenters[cityLabels[bs_id]] = (latitude + coords[0], longitude + coords[1])

for cityLabel in cityLabels.unique():
    cityCenters[cityLabel] = cityCenters.get(cityLabel) / cityBsStationCount[cityLabel]

In [ ]:
def rgb_to_hex(rgb):
        return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def labelToColor(clustId, clusters, palette):   
    if (clustId == -1):
        return 'gray'
    else:
        return rgb_to_hex(palette[list(clusters).index(clustId)- (1 if -1 in clusters else 0)])

In [ ]:
clusters = cityLabels.unique()
num_clusters = len(clusters) - (1 if -1 in clusters else 0)
palette = sns.color_palette("hsv", num_clusters)
colors = cityLabels.apply(lambda clustId : labelToColor(clustId, clusters, palette))

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']].loc[cityStationsLabels], axis=0), zoom_start=7, tiles="Cartodb Positron")
bigCitiesLayer = folium.FeatureGroup(f"Big cities").add_to(map)
littleCitiesLayer = folium.FeatureGroup(f"Little cities").add_to(map)
bigCityCentersLayer = folium.FeatureGroup(f"big cities centers").add_to(map)
litleCityCenterLayer = folium.FeatureGroup(f"little cities centers").add_to(map)
for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].loc[cityStationsLabels].itertuples():
    color = colors[bs_id]
    dot = folium.CircleMarker(location=[latitude, longitude], color=color, radius=1, popup=cityLabels[bs_id])

    if(cityLabels[bs_id] in bigCitiesClusters):
        bigCitiesLayer.add_child(dot)
    else:
        littleCitiesLayer.add_child(dot)

for cityLabel in cityLabels.unique():
    
    if(cityLabel in bigCitiesClusters):
        dot = folium.CircleMarker(location=cityCenters[cityLabel], color='black', fill_opacity = 1, fill=True, radius=7, popup=cityBsStationCount[cityLabel])
        bigCityCentersLayer.add_child(dot)
    else:
        dot = folium.CircleMarker(location=cityCenters[cityLabel], color='black', fill_opacity = 1, fill=True, radius=2, popup=cityBsStationCount[cityLabel])
        litleCityCenterLayer.add_child(dot)

folium.LayerControl().add_to(map)

map.save(f"../../out/maps/SeparateLilFromBigCities.html")

In [ ]:
def conformEdge(edge, pos):
    return km_distance(pos.get(edge[0]), pos.get(edge[1])) <= 500

In [ ]:
G = nx.Graph()

for cluster in bigCitiesClusters:
    G.add_node(cluster, pos=cityCenters.get(cluster))

pos = nx.get_node_attributes(G, 'pos')  

points_pos_big = [cityCenters.get(cluster) for cluster in bigCitiesClusters]
delaunay_triangulation = Delaunay(points_pos_big)


for simplex in delaunay_triangulation.simplices:
    edgesToAdd = [(bigCitiesClusters[start], bigCitiesClusters[end]) for (start, end) in combinations(simplex, 2)]
    G.add_edges_from(edgesToAdd)

reversed_positions = {node_id: (pos[1], pos[0]) for node_id, pos in pos.items()}

G = distance_criterion(G, pos, max_distance=400)
G = angle_criterion(G, pos, min_angle=15)

nx.draw(G, reversed_positions)

In [ ]:
def distance_to_segment(A, B, C):
    A = np.array(A)
    B = np.array(B)
    C = np.array(C)
    
    P = A + (np.dot(B-A, C-A))/(np.linalg.norm(B-A)**2) * (B-A)
    
    k = np.dot(B-A, P-A) / np.linalg.norm(B-A)**2

    dist = np.linalg.norm(P-C) if ((k >= 0) and (k <= 1)) else np.inf
    
    return dist, P

In [ ]:
edgesInBetween = {}
for [sEdge, eEdge] in G.edges :
    inBetween = []
    for cityId in littleCitiesClusters :
        dist, P = distance_to_segment(cityCenters[sEdge], cityCenters[eEdge], cityCenters[cityId])
        if (dist <= 0.15) :
            inBetween.append(cityId)
    edgesInBetween[(sEdge, eEdge)] = inBetween

In [ ]:
G_complete = nx.Graph()
for cluster in bigCitiesClusters:
    G_complete.add_node(cluster, pos=cityCenters.get(cluster))

for [sEdge, eEdge] in G.edges:
    edges = edgesInBetween.get((sEdge, eEdge))
    for cluster in edges:
        G_complete.add_node(cluster, pos=cityCenters.get(cluster))

    edgesToReach = set(edges)
    startingEdge = sEdge

    while (len(edgesToReach) != 0):
        startingPos = np.array(cityCenters[startingEdge])
        dist = np.inf
        nextEdge = startingEdge
        for possibleEdge in edgesToReach:
            possibleEdgePos = np.array(cityCenters[possibleEdge])
            possibleDist = np.linalg.norm(possibleEdgePos-startingPos)
            if(possibleDist <= dist):
                nextEdge = possibleEdge
                dist = possibleDist
        edgesToReach.remove(nextEdge)
        G_complete.add_edge(startingEdge, nextEdge)
        startingEdge = nextEdge
    G_complete.add_edge(startingEdge, eEdge)


pos = nx.get_node_attributes(G_complete, 'pos')  

reversed_positions = {node_id: (pos[1], pos[0]) for node_id, pos in pos.items()}

node_sizes = [cityBsStationCount.loc[node] for node in G_complete.nodes]

nx.draw(G_complete, reversed_positions, node_size = node_sizes)

In [ ]:
possibleRoadBsStations = road_candidates(df_extracted)

roadBsStations = set()
for edge in G_complete.edges:
    for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].loc[possibleRoadBsStations].itertuples():
        dist, P = distance_to_segment(cityCenters[edge[0]], cityCenters[edge[1]], [latitude, longitude])

        if (dist <= 0.4):
            roadBsStations.add(bs_id)

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']].loc[cityStationsLabels], axis=0), zoom_start=7, tiles="OpenStreetMap")
SelectedPossibleRoadsLayer = folium.FeatureGroup(f"Selected possible roads").add_to(map)
UnselectedPossibleRoadsLayer = folium.FeatureGroup(f"Unselected possible roads").add_to(map)
graphLayer = folium.FeatureGroup(f"Graph").add_to(map)
for bs_id, latitude, longitude in df_extracted[['latitude', 'longitude']].loc[possibleRoadBsStations].itertuples():
    dot = folium.CircleMarker(location=[latitude, longitude], color='blue', fill_opacity = 1, fill=True, radius=3)
    if (bs_id in roadBsStations) :
        dot.add_to(SelectedPossibleRoadsLayer)
    else : 
        dot.add_to(UnselectedPossibleRoadsLayer)

for edge in G_complete.edges:
    folium.PolyLine(locations=np.vstack([cityCenters[edge[0]], cityCenters[edge[1]]])).add_to(graphLayer)

folium.LayerControl().add_to(map)
map.save(f"../../out/maps/road.html")